In [3]:
import os

#an import statement - you know to import something
from google.cloud import bigquery

# Fetch Credentials
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = 'local-receiver-341803-9e38525e7b40.json'

# Build BigQuery Client
client = bigquery.Client()

In [4]:
pwd

'/content'

In [6]:
import pandas as pd

# Query for Position Histories from BigQuery
QUERY = ('SELECT lastTrainReporting.positionHistoryList FROM `local-receiver-341803.testTrainEvents.train-event-data` LIMIT 1')
query_job = client.query(QUERY)

# Wait for job to finish
rows = query_job.result()

# Fetch each position time and milepost
positionMilepostTimesDataFrame = pd.DataFrame()

# For each returned record
for row in rows:
    # Scan each position history
    for positionHistory in row.get('positionHistoryList'):
        # The time that the train was at the specific milepost
        # Record the position time as seconds from epoch to use in regression model
        positionTime = positionHistory.get('positionTime').timestamp()

        # How fast the train is traveling
        speedMPH = positionHistory.get('speedMPH')

        # Milepost location information
        milepostLocation = positionHistory.get('milepostLocation')
        # The unique identifier of the subdivision
        subdivisionId = milepostLocation.get('subdivisionId')
        # The unique identifier fo the milepost
        milepostNumber = milepostLocation.get('milepost').get('milepostNumber')

        # Construct new dataframe record
        dataframeRow = {'PositionTime': positionTime, 'SpeedMPH': speedMPH, 
                        'SubdivisionId': subdivisionId, 'MilepostNumber':milepostNumber}

        # Append the row to the data frame.
        positionMilepostTimesDataFrame = positionMilepostTimesDataFrame.append(dataframeRow, ignore_index = True)

positionMilepostTimesDataFrame


,PositionTime,SpeedMPH,SubdivisionId,MilepostNumber
0,1.648865e+09,-11.0,106.0,766.9261
1,1.648864e+09,-10.0,595.0,279.0362
2,1.648864e+09,-9.0,595.0,279.1244
3,1.648864e+09,-7.0,595.0,279.2209
4,1.648864e+09,-3.0,595.0,279.3100


In [8]:
from sklearn.linear_model import LinearRegression
import numpy as np

# Fetch the position time
X = np.array(positionMilepostTimesDataFrame)[:,1].reshape(-1,1)
y = np.array(positionMilepostTimesDataFrame.drop('PositionTime',axis=1)).reshape(-1, 3)

# Fit the linear regression model
regsr=LinearRegression()
regsr.fit(X,y)

LinearRegression()

In [9]:
import datetime

# Build predicition set
to_predict_x = [datetime.datetime(2022, 3, 31, 17, 20, 23, tzinfo=datetime.timezone.utc).timestamp()]
to_predict_x = np.array(to_predict_x).reshape(-1,1)

# Predict 
predicted_y = regsr.predict(to_predict_x)

m = regsr.coef_
c = regsr.intercept_

print("Predicted y:\n", predicted_y)
print("slope (m): ", m)
print("y-intercept (c): ", c)


Predicted y:
 [[ 1.64874722e+09  6.04678052e+10 -6.02701278e+10]]
slope (m):  [[  1.      ]
 [ 36.675   ]
 [-36.555105]]
y-intercept (c):  [  0.      790.6      84.28268]
